<a href="https://colab.research.google.com/github/Muhammadsulton1/MIPT_Data_analys/blob/main/%D0%B4%D0%BE%D0%B2%D0%B5%D1%80%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D0%B0%D0%BB%D1%8B_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

<center>
<img src="logo.png" height="900">
</center>


#  Точные доверительные интервалы

Будем работать с данными по стоимости квартир в москве из таблички `flat.csv`.


In [ ]:
def norm_conf_int(alpha, mean_hat, std_hat, margin=5):
    """
        Строит 95% асимптотически-нормальный доверительный интервал
    """

    plt.figure(figsize=(10,5))
    xs = np.linspace(mean_hat - margin, mean_hat + margin)
    pdf = stats.norm(mean_hat, std_hat).pdf(xs)

    plt.plot(xs, pdf)
    plt.ylabel('$f(x)$', fontsize=18)
    plt.xlabel('$x$', fontsize=18)

    left, right = stats.norm.interval(1 - alpha, loc=mean_hat, scale=std_hat)

    for i in [left, right]:
        y_max = plt.ylim()[1]
        plt.axvline(i, color="blue", linestyle='dashed', lw=2)

        if i == left:
            xq = np.linspace(mean_hat - margin, left)
        else:
            xq = np.linspace(right, mean_hat + margin)

        text_margin = 0.05
        plt.text(i + text_margin, 0.8*y_max, round(i, 2), color="blue", fontsize=14)
        yq = stats.norm(mean_hat, std_hat).pdf(xq)
        plt.fill_between(xq, 0, yq, color='blue', alpha=0.3)

    return left, right

__Описание переменных:__

```
n – номер квартиры по порядку
price – цена квартиры в $1000
totsp – общая площадь квартиры, кв.м.
livesp жилая площадь квартиры, кв.м.
kitsp – площадь кухни, кв.м.
dist – расстояние от центра в км.
metrdist – расстояние до метро в минутах
walk – 1 – пешком от метро, 0 – на транспорте
brick 1 – кирпичный, монолит ж/б, 0 – другой
floor 1 – этаж кроме первого и последнего, 0 – иначе.
code – число от 1 до 8, при помощи которого мы группируем наблюдения по
подвыборкам:
1. Наблюдения сгруппированы на севере, вокруг Калужско-Рижской линии
метрополитена
2. Север, вокруг Серпуховско-Тимирязевской линии метрополитена
3. Северо-запад, вокруг Замоскворецкой линии метрополитена
4. Северо-запад, вокруг Таганско-Краснопресненской линии метрополитена
5. Юго-восток, вокруг Люблинской линии метрополитена
6. Юго-восток, вокруг Таганско-Краснопресненской линии метрополитена
7. Восток, вокруг Калиниской линии метрополитена
8. Восток, вокруг Арбатско-Покровской линии метрополитена
```


In [ ]:
df = pd.read_csv('flat.csv', sep='\t')
print(df.shape)
df.head()

# 1. Доверительные интервалы для среднего

Построим $95\%$ асимптотический доверительный интервал для средней стоимости квартиры. Построим точный доверительный интервал для средней стоимости квартиры. Какой из них оказался уже? Почему?

__Асимптотический доверительный интервал:__

$$
\bar x \pm z_{1 - \frac{\alpha}{2}} \cdot \sqrt{\frac{\hat s^2}{n}}.
$$

> __Предположения:__
В выборке нет аномалий, собиралась независимо, тогда среднее асимптотически нормально распределено (ЦПТ)

In [ ]:
# вручную
norm_rv = stats.norm()

alpha = 0.05
z_crit = norm_rv.ppf(1 - alpha/2)

mu_hat = df.price.mean()
var_hat = df.price.var(ddof=1)
n = df.price.count()

left = mu_hat - z_crit*np.sqrt(var_hat/n)
right = mu_hat + z_crit*np.sqrt(var_hat/n)

print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))

In [ ]:
mu_hat = df.price.mean()
sd_hat = df.price.std(ddof=1)/np.sqrt(n)

stats.norm.interval(0.95, loc=mu_hat, scale=sd_hat)

In [ ]:
alpha = 0.05
mu_hat = df.price.mean()
sd_hat = df.price.std(ddof=1)/np.sqrt(n)

norm_conf_int(alpha, mu_hat, sd_hat, margin=5)

__Точный доверительный интервал:__

$$
\bar x \pm t_{1 - \frac{\alpha}{2}} \cdot \sqrt{\frac{\hat s^2}{n}}.
$$

> __Предположения:__
Выборка пришла из нормального распределения, дисперсия неизвестна

In [ ]:
# вручную
n = df.price.count()
t_rv = stats.t(n - 1)

alpha = 0.05
t_crit = t_rv.ppf(1 - alpha/2)

mu_hat = df.price.mean()
var_hat = df.price.var(ddof=1)


left = mu_hat - t_crit*np.sqrt(var_hat/n)
right = mu_hat + t_crit*np.sqrt(var_hat/n)

print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))

In [ ]:
n

In [ ]:
n
# Оказался немного шире, но похож, так как:
# t(n) => N(0,1) при n -> inf

In [ ]:
# пакетом
n = df.price.count()
mu_hat = df.price.mean()
sd_hat = df.price.std(ddof=1) / np.sqrt(n)

# тут 0.95 это 1 - alpha
left, right = stats.t.interval(0.95, df=(n-1), loc=mu_hat, scale=sd_hat)
print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))


> Видим, что интервалы не сильно отличаются друг от друга, из-за того, что у нас в выборке много наблюдений. При большом числе степеней свободы распределение Стьюдента похоже на нормальное. Тем не менее даже при большом количестве наблюдений точный доверительный интервал оказывается немного шире.

__Построим $99\%$ точные доверительные интервалы для всех районов. В какие из них попало общее среднее? Построим красивую визулизацию.__

In [ ]:
df.head()

In [ ]:
df_agg = df.groupby('code')['price'].agg(['mean', 'std', 'count'])
df_agg

In [ ]:
alpha = 0.01
df_agg['t_crit'] = df_agg['count'].apply(lambda w: stats.t(w - 1).ppf(1 - alpha/2))

# на сколько отступать вправо и влево от математического ожидания
df_agg['step'] =  df_agg['t_crit']  * df_agg['std']/np.sqrt(df_agg['count'])
df_agg

In [ ]:
district = [
    'Север, Калужско-Рижская', 'Север, Серпуховско-Тимирязевская',
    'Северо-запад, Замоскворецкая', 'Северо-запад, Таганско-Краснопресненская',
    'Юго-восток, Люблинская', 'Юго-восток, Таганско-Краснопресненская',
    'Восток, Калиниская', 'Восток, Арбатско-Покровская'
]

center = df_agg['mean'].values
step = df_agg['step'].values

mu_hat = df.price.mean() # общее среднее

plt.figure(figsize=(10,5))

plt.errorbar(center, np.arange(center.size),  xerr = step,
             capsize=0, fmt="o", color="blue")

plt.yticks(np.arange(center.size - 1, -1, -1), district, fontsize=12);
plt.axvline(mu_hat, 0, color='grey');

__Выводы:__

- В среднем, самая дорогая недвижимость находится на Юго-востоке. Самая дешёвая на Северо-западе.

# 2. Доверительный интервал для разности средних

Построим $95\%$ асимптотический доверительный интервал для разницы в средней стоимости квартир в монолитных и панельных домах.

$$
\bar x - \bar y \pm z_{crit} \cdot \sqrt{\frac{\hat\sigma_x^2}{n_x} + \frac{\hat\sigma_y^2}{n_y}}
$$

In [ ]:
x = df[df.brick == 1].price.values
y = df[df.brick == 0].price.values

diff = x.mean() - y.mean()
nx,ny = x.size, y.size
diff_sd = np.sqrt(x.var(ddof=1)/nx + y.var(ddof=1)/nx)

left, right = stats.norm.interval(0.95, loc=diff, scale=diff_sd)
print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))

Предположим, что дисперсии неизвестны, но равны.

$$
\bar x - \bar y \pm t_{crit} \cdot \sqrt{\frac{s^2}{n_x} + \frac{s^2}{n_y}}
$$

In [ ]:
diff = x.mean() - y.mean()
nx, ny = x.size, y.size

s2 = ((nx - 1)*x.var() + (ny - 1)*y.var())/(nx + ny - 2)
diff_sd = np.sqrt(s2/nx + s2/ny)

left, right = stats.t.interval(0.95, df=(nx + ny - 2), loc=diff, scale=diff_sd)
print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))

In [ ]:
x.var(), y.var()

Предположим, что дисперсии неизвестны и не равны. Построим приближённый доверительный интервал (интервал Уэлча).

In [ ]:
diff = x.mean() - y.mean()
nx, ny = x.size, y.size
diff_sd = np.sqrt(x.var()/nx + y.var()/ny)

u = (x.var() / nx + y.var() / ny) ** 2
d = (x.var()**2) / (nx**2 * (nx - 1)) + (y.var()**2)/ (ny**2* (ny - 1))
nu =  u/d
print("Число степеней свободы:", nu)

left, right = stats.t.interval(0.95, df=nu, loc=diff, scale=diff_sd)
print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))


In [ ]:
nx, ny

In [ ]:
x.std(), y.std()  # Знаки не в ту сторону => при малых выборках не очень верим

> Все три доверительных интервала из-за больших объёмов выборки снова оказались похожи. Выборочные дисперсии довольно сильно отличаются друг от друга, из-за этого использование доверительного интервала с предпосылкой о их равенстве некорректно.

__Выводы:__

- В среднем, на уровне значимости $5\%$, квартиры в монолитных домах стоят дороже, чем в панельных. Ноль нигде не попадает в доверительный интервал.

# 3. Доверительный интервал для дисперсии

Предполагая нормальность распределения цен, построим $95\%$ доверительный интервал для дисперсии стоимости квартир.

$$
\frac{(n - 1) \cdot s^2}{\chi_{n-1}^2 \left(1 - \frac{\alpha}{2} \right)} \le \sigma^2 \le \frac{(n - 1) \cdot s^2}{\chi_{n-1}^2 \left(\frac{\alpha}{2} \right)}
$$


> __Предположения:__
Выборка пришла из нормального распределения, математическое ожидание неизвестно, выполняется теорема Фишера


In [ ]:
alpha = 0.05

n = df.price.count()
var_hat = df.price.var(ddof=1)

chi_l, chi_u = stats.chi2.ppf([1-alpha/2, alpha/2], df = n - 1)

left = (n - 1) * var_hat/chi_l
right = (n - 1) * var_hat/chi_u

print("Доверительный интервал [{:.5}; {:.5}] ширины {:.5}".format(left, right, right - left))


Сделаем то же самое, но в разбивке по районам.

In [ ]:
df_agg = df.groupby('code')['price'].agg(['var', 'count'])

alpha = 0.05
df_agg['ch_l'] = df_agg['count'].apply(lambda w: stats.chi2.ppf(1-alpha/2, df = w - 1))
df_agg['ch_u'] = df_agg['count'].apply(lambda w: stats.chi2.ppf(alpha/2, df = w - 1))

df_agg['left'] =  (df_agg['count'] - 1) * df_agg['var']/df_agg['ch_l']
df_agg['right'] =  (df_agg['count'] - 1) * df_agg['var']/df_agg['ch_u']

df_agg

In [ ]:
district = [
    'Север, Калужско-Рижская', 'Север, Серпуховско-Тимирязевская',
    'Северо-запад, Замоскворецкая', 'Северо-запад, Таганско-Краснопресненская',
    'Юго-восток, Люблинская', 'Юго-восток, Таганско-Краснопресненская',
    'Восток, Калиниская', 'Восток, Арбатско-Покровская'
]

center = df_agg['var'].values
step = np.array([df_agg['var'].values - df_agg['left'].values,
                 df_agg['right'].values - df_agg['var'].values])

var_hat = df.price.var()

plt.figure(figsize=(10,5))

plt.errorbar(center, np.arange(center.size),  xerr = step,
             capsize=0, fmt="o", color='blue')

plt.yticks(np.arange(center.size - 1, -1, -1), district, fontsize=12);
plt.axvline(var_hat, 0, color='grey');

__Выводы:__

- На Юго-Востоке самый большой разброс в ценах. Как покупатель, я найду здесь более разнообразные предложения. Так происходит из-за того, что с одной стороны районы близки к центру и здесь есть респектабельная недвижимость. С другой стороны, в районах довольно много советской застройки с плохой планировкой и ремонтом.

# 4. Доверительный интервал для отношения дисперсий

Предполагая нормальность распределения, построим $95\%$-ый доверительный интервал для отношения дисперсии стоимости квартир в монолитных домах и в панельных домах.


$$
\frac{s^2_m}{s^2_n} \cdot F_{\frac{\alpha}{2}}(n,m) \le \frac{\sigma^2_m}{\sigma^2_n} \le  \frac{s^2_m}{s^2_n} \cdot F_{1 - \frac{\alpha}{2}}(n,m)
$$

In [ ]:
alpha = 0.05

x = df[df.brick == 1].price.values
y = df[df.brick == 0].price.values

n, m = x.size - 1, y.size - 1
sn, sm = x.var(), y.var()

left = sm / sn * stats.f(n, m).ppf(alpha/2)
right = sm / sn * stats.f(n, m).ppf(1 - alpha/2)

print("Доверительный интервал [{:.5}; {:.5}] ширины {:.5}".format(left, right, right - left))

In [ ]:
x.std(), y.std()

__Выводы:__

- Единица не попала в доверительный интервал. Дисперсии в ценах на монолитное жильё и на панельное различаются. Цены на монолитное жильё обладают более высокой дисперсией.

## А можно ли было пользоваться нормальным распределением?

Распределение цен выглядит не очень нормальным.

In [ ]:
df.price.hist(bins=50,density=True);

Распределение логарифмов цен оказывается более похожим на нормальное.

In [ ]:
df.price.apply(np.log).hist(bins=50, density=True);

x = df.price.apply(np.log).values

xs = np.linspace(3.5, 7, 300)
ys = stats.norm(loc=np.mean(x), scale=np.std(x, ddof=1)).pdf(xs)
plt.plot(xs, ys, color="black");

Можно формально проверить гипотезу о том, имеет ли какая-то выборка нормальное распределение с помошью теста Колмогорова-Смирнова. О нём мы подробнее поговорим в будущем, а сейчас просто воспользуемся.

In [ ]:
stats.kstest(df.price, 'norm', args=(df.price.mean(), df.price.std(ddof=1)))

In [ ]:
stats.kstest(x, 'norm', args=(np.mean(x), np.std(x, ddof=1)))

В обоих ситуациях `pvalue` очень низкое, это говорит о том, что выборка не распределена нормально.

<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars/sem_5/pEbNtDxzx0Q.jpg" height="500" width="500">